This notebook aims to get cluster of questions.

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

from sklearn.cluster import KMeans

In [ ]:
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
questions.head()
questions.shape

In [ ]:
questions = questions[~questions['tags'].isna()]
questions.shape

In [ ]:
questions.head()

In [ ]:
questions = questions.assign(tags=questions['tags'].str.split()).explode('tags')
questions.head()

check tag num

In [ ]:
print(f"total tag num is {len(questions['tags'].value_counts())}.")

In [ ]:
questions = questions[['question_id', 'tags']]
df = pd.get_dummies(questions, columns=['tags'])
df.head()

In [ ]:
tmp_df = df.groupby('question_id').agg(['sum'])
tmp_df.head()

In [ ]:
km = KMeans(n_clusters=20,
            init='random',
            n_init=10,
            random_state=0)

In [ ]:
X = tmp_df.iloc[:, 1:]
cluster_labels = km.fit_predict(X)
cluster_labels

In [ ]:
tmp_df['k'] = cluster_labels
tmp_df = tmp_df.reset_index()
tmp_df = tmp_df[['question_id', 'k']]
tmp_df.columns = tmp_df.columns.droplevel(1)
tmp_df.head(10)

Get best cluster num using silhouette score.

In [ ]:
from sklearn.metrics import silhouette_score

scores = []
for i in range(5, 100, 10):
    km = KMeans(n_clusters=i,
                init='random',
                n_init=10,
                random_state=0)
    cluster_labels = km.fit_predict(X)
    score = silhouette_score(X, cluster_labels)
    scores.append(score)
    print(f'cluster num: {i}, silhouette score: {score}')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')

fig = plt.figure(figsize=(12,6))
pd.Series(scores, index=list(range(5, 100, 10))).plot.bar(scores);